In [ ]:
import pandas as pd
from databakerUtils.api import getAllCodes, getAllLabels

input_file = "Mid-year estimates 2012 to 2017 for the UK - backseries from SAS.xlsx"
output_file = "v4_mid_year_pop_est.csv"

source = pd.read_excel(input_file)

#create a dict from the api geography codelist
url = 'https://api.beta.ons.gov.uk/v1/code-lists/admin-geography/editions/one-off/codes'
dataCodes = getAllCodes(url) 
dataLabels = getAllLabels(url) 
geoDict = dict(zip(dataCodes,dataLabels))

def geoLookup(value):
    return geoDict[value]

def sexLookup(value):
    sex = {
            0:"All",
            1:"Male",
            2:"Female"
            }
    return sex[value]

dataFrameList = []

for i in range(3,len(source.columns)):
    
    time = source.columns[i]
    
    dfLoop = pd.DataFrame()
    
    dfLoop['V4_0'] = source[time]
    
    dfLoop['calendar-years'] = time[-4:]
    dfLoop['time'] = time[-4:]
    
    dfLoop['admin-geography'] = source['LAD_code']
    dfLoop['geography'] = source['LAD_code'].apply(geoLookup)
    
    dfLoop['mid-year-pop-age'] = source['age']
    dfLoop['age'] = source['age']
    
    dfLoop['mid-year-pop-sex'] = source['Sex'].apply(lambda x:str(x))
    dfLoop['sex'] = source['Sex'].apply(sexLookup)
    
    dataFrameList.append(dfLoop)
    
df = pd.concat(dataFrameList)

#changing "max age" to "max age +" -- "90" to "90+"
ageList = list(df['age'].unique())
ageList.remove('Total')
maxAge = ageList[-1]
df['age'] = df['age'].apply(lambda x:x.replace(maxAge,maxAge+"+"))
df['mid-year-pop-age'] = df['age'].apply(lambda x:x.lower())

#there are two 'west midlands' - changing the lower level one
df.loc[df['admin-geography'] == 'E11000005','geography'] = 'West Midlands (Met County)'

df.to_csv(output_file,index=False)